In [14]:
url1 = (r'https://raw.githubusercontent.com/owid/covid-19-data/master/public/data/latest/owid-covid-latest.json')

In [3]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
import numpy as np
import requests, pandas as pd
from bs4 import BeautifulSoup
import urllib.request

In [67]:

#This is live data from the our world in data github. This is the csv they put on the website. Covid19 data.
data=pd.read_json(url1)
data1 = pd.DataFrame(data).T
data1 = data1.astype(str)
data1 = data1[data1 != 'None']
data1 = data1['total_deaths'].astype(float)


In [73]:
url2 = ('https://apps.who.int/gho/athena/api/GHO/HWF_000.json?filter=YEAR:2019&filter=text')
url2

'https://apps.who.int/gho/athena/api/GHO/HWF_000.json?filter=YEAR:2019&filter=text'

In [75]:
data2 = pd.read_csv('C:\Users\vw191\OneDrive\Documents\GitHub\107SBakst.github.io\data\who_doctors.csv')

SyntaxError: (unicode error) 'unicodeescape' codec can't decode bytes in position 2-3: truncated \UXXXXXXXX escape (<ipython-input-75-e5711215ba4b>, line 1)